In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/val/val_week1_2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/test/test_week2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/clean_data_week2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/val/val_week1_2_3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/test/test_week3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/clean_data_week3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw

## 1. Import libraries

In [2]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.utils import to_categorical
import xgboost as xgb
from kerastuner.tuners import RandomSearch
import os
from collections import defaultdict
from imblearn.over_sampling import SMOTE
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tensorflow.keras.utils import to_categorical

<ipython-input-2-5ad9f3258c65>:15: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


## 2. Load dataset

In [3]:
five_fold_files = {
    "week1": [
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_5.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_5.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/5-folds/data_part_5.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week4/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week4/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week4/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week4/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week4/train/5-folds/data_part_5.csv"
    ]
}

file_validataion = {
    "week1": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/val/val_week1.csv",
    "week2": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/val/val_week1_2.csv",
    "week3": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/val/val_week1_2_3.csv",
    "week4": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/val/val_week1_2_3_4.csv"
}

file_test = {
    "week1": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/test/test_week1.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/test/test_week2.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/test/test_week3.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/test/test_week4.csv"
    ]
}

## Mô hình FCNN

## Dự đoán các week  với best parameters

In [4]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình
def build_model(params):
    model = tf.keras.Sequential()
    
    # Thêm các lớp ẩn dựa trên số lượng lớp
    for i in range(params['num_layers']):
        units = params.get(f'units_{i}', 32)  # Mặc định là 32 nếu không có
        dropout = params.get(f'dropout_{i}', 0.2)  # Mặc định là 0.2 nếu không có
        model.add(layers.Dense(units=units, activation='relu'))
        model.add(layers.Dropout(rate=dropout))
    
    # Lớp đầu ra
    model.add(layers.Dense(5, activation='softmax'))  # 5 lớp output
    
    # Compile mô hình
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
        loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy']
    )
    
    return model

In [5]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": {
        "num_layers": 3,
        "units_0": 160,
        "dropout_0": 0.2,
        "units_1": 128,
        "dropout_1": 0.4,
        "learning_rate": 0.0010624691555580386,
        "units_2": 64,
        "dropout_2": 0.2,
        "units_3": 96,
        "dropout_3": 0.2,
        "units_4": 192,
        "dropout_4": 0.1
    },
    "week2": {
        "num_layers": 4,
        "units_0": 64,
        "dropout_0": 0.3,
        "units_1": 160,
        "dropout_1": 0.3,
        "learning_rate": 0.0002972054749315651,
        "units_2": 128,
        "dropout_2": 0.1,
        "units_3": 192,
        "dropout_3": 0.3,
        "units_4": 256,
        "dropout_4": 0.3
    },
    "week3": {
        "num_layers": 2,
        "units_0": 128,
        "dropout_0": 0.3,
        "units_1": 32,
        "dropout_1": 0.1,
        "learning_rate": 0.000580266006484294,
        "units_2": 64,
        "dropout_2": 0.4,
        "units_3": 224,
        "dropout_3": 0.1,
        "units_4": 224,
        "dropout_4": 0.5
    },
    "week4": {
        "num_layers": 3,
        "units_0": 160,
        "dropout_0": 0.1,
        "units_1": 128,
        "dropout_1": 0.4,
        "learning_rate": 0.0004542805978770659,
        "units_2": 160,
        "dropout_2": 0.1,
        "units_3": 32,
        "dropout_3": 0.2,
        "units_4": 256,
        "dropout_4": 0.2
    }
}

## 5-folds

In [6]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week]
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)
        
        # Xây dựng mô hình với tham số tốt nhất
        model = build_model(params)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["confusion_matrices"].append(conf_matrix)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.mean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.mean(week_results["f1_score_per_label"], axis=0)
    average_confusion_matrix = np.mean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    
    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    
    # In kết quả
    print("\n=== Average Precision, Recall, F1-Score per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)

    # Lưu kết quả tuần vào kết quả tổng quát
    week_results["average_train_times"] = average_train_time
    week_results["average_test_times"] = average_test_time
    week_results["average_metrics_df"] = metrics_df
    week_results["average_confusion_matrix"] = confusion_df
    overall_results_5folds.append(week_results)



Processing week1 with best parameters...
best parameters for week1: {'num_layers': 3, 'units_0': 160, 'dropout_0': 0.2, 'units_1': 128, 'dropout_1': 0.4, 'learning_rate': 0.0010624691555580386, 'units_2': 64, 'dropout_2': 0.2, 'units_3': 96, 'dropout_3': 0.2, 'units_4': 192, 'dropout_4': 0.1}
Fold 1: Using file /kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6026 - loss: 1.5864 - val_accuracy: 0.6146 - val_loss: 1.5450
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6136 - loss: 1.5394 - val_accuracy: 0.6115 - val_loss: 1.5654
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6103 - loss: 1.5704 - val_accuracy: 0.6115 - val_loss: 1.5654
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6110 - loss: 1.5675 - val_accuracy: 0.6115 - val_loss: 1.5654
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6031 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5845 - loss: 1.5648 - val_accuracy: 0.6714 - val_loss: 1.2788
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6336 - loss: 1.3176 - val_accuracy: 0.6466 - val_loss: 0.1649
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5551 - loss: 0.2514 - val_accuracy: 0.6420 - val_loss: 0.1422
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6130 - loss: 0.1559 - val_accuracy: 0.6390 - val_loss: 0.1326
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6307 - loss: 0.1415 - val_accuracy: 0.6512 - val_loss: 0.1308
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6408 - loss: 0.1339 - val_accuracy: 0.6580 - val_loss: 0.1269
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6438 - loss: 0.1340 - val_accuracy: 0.6618 - val_loss: 0.1263
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6478 - loss: 0.1327 - val_accuracy: 0.6615 - val_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.1846 - loss: 3.2053 - val_accuracy: 0.6278 - val_loss: 1.4662
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6306 - loss: 1.4117 - val_accuracy: 0.6834 - val_loss: 1.1727
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6601 - loss: 1.1891 - val_accuracy: 0.7071 - val_loss: 0.8189
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6140 - loss: 0.6071 - val_accuracy: 0.6217 - val_loss: 0.1529
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6254 - loss: 0.1725 - val_accuracy: 0.6175 - val_loss: 0.1310
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6278 - loss: 0.1413 - val_accuracy: 0.6186 - val_loss: 0.1306
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6173 - loss: 0.1399 - val_accuracy: 0.6163 - val_loss: 0.1302
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6155 - loss: 0.1371 - val_accuracy: 0.6159 - val_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4725 - loss: 1.7769 - val_accuracy: 0.5011 - val_loss: 0.2003
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5702 - loss: 0.3331 - val_accuracy: 0.6194 - val_loss: 0.1377
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6177 - loss: 0.1785 - val_accuracy: 0.6186 - val_loss: 0.1280
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6318 - loss: 0.1439 - val_accuracy: 0.6201 - val_loss: 0.1264
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6221 - loss: 0.1413 - val_accuracy: 0.6201 - val_loss: 0.1248
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6173 - loss: 0.1383 - val_accuracy: 0.6236 - val_loss: 0.1249
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6181 - loss: 0.1369 - val_accuracy: 0.6243 - val_loss: 0.1248
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6231 - loss: 0.1341 - val_accuracy: 0.6274 - val_

## Kết quả 5 folds

In [7]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_confusion_matrix = week_result["average_confusion_matrix"]

    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print("\nAverage Precision, Recall, F1-Score per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Train Time: 48.9190 seconds
Average Test Time: 0.2907 seconds

Average Precision, Recall, F1-Score per Label:
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.650193        0.566000          0.598587
1      1           0.700000        0.029650          0.056308
2      2           0.621891        0.212720          0.313606
3      3           0.593336        0.201198          0.298175
4      4           0.753128        0.938003          0.834318

Average Confusion Matrix:
       0    1     2     3       4
0  339.6  0.4   2.6   1.0   256.4
1   29.2  2.6   0.4   1.8    53.6
2   37.8  0.0  35.0   5.4    86.2
3   26.2  0.0   2.8  33.6   104.4
4   91.4  0.0   4.4   3.6  1504.0

=== Results for week2 ===
Average Train Time: 58.0210 seconds
Average Test Time: 0.3106 seconds

Average Precision, Recall, F1-Score per Label:
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.697543        0.618000  

## Kết quả trên tập test

In [8]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_test = test_data["classification_encoded"]
    
    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)
    
    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=5)
    
    # Lấy tham số tốt nhất cho tuần
    params = best_params[f"week{week_num}"]
    
    # Xây dựng mô hình
    model = build_model(params)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_res, y_train_res, epochs=50, validation_split=0.1, batch_size=32, verbose=1)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    
    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix
    })
    
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [9]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.3006 - loss: 2.1634 - val_accuracy: 0.0000e+00 - val_loss: 0.3567
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3290 - loss: 0.2363 - val_accuracy: 0.0092 - val_loss: 0.3490
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3544 - loss: 0.2239 - val_accuracy: 0.0841 - val_loss: 0.3187
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3872 - loss: 0.2176 - val_accuracy: 0.3477 - val_loss: 0.3299
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4234 - loss: 0.2098 - val_accuracy: 0.4802 - val_loss: 0.2947
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4588 - loss: 0.2023 - val_accuracy: 0.4475 - val_loss: 0.2692
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4735 - loss: 0.1987 - val_accuracy: 0.3754 - val_loss: 0.2811
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - acc

In [10]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.2677 - loss: 1.8901 - val_accuracy: 0.0377 - val_loss: 0.2940
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.3201 - loss: 0.2539 - val_accuracy: 0.0771 - val_loss: 0.3114
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3800 - loss: 0.2262 - val_accuracy: 0.0619 - val_loss: 0.3072
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4148 - loss: 0.2152 - val_accuracy: 0.0995 - val_loss: 0.3013
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4384 - loss: 0.2081 - val_accuracy: 0.1664 - val_loss: 0.2681
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4563 - loss: 0.2013 - val_accuracy: 0.5009 - val_loss: 0.2643
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4836 - loss: 0.1950 - val_accuracy: 0.5228 - val_loss: 0.2348
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accurac

In [11]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.3195 - loss: 2.1775 - val_accuracy: 2.4944e-04 - val_loss: 0.3120
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2908 - loss: 0.2517 - val_accuracy: 0.0035 - val_loss: 0.3582
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2982 - loss: 0.2388 - val_accuracy: 0.0561 - val_loss: 0.3699
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3153 - loss: 0.2329 - val_accuracy: 0.1152 - val_loss: 0.3545
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3424 - loss: 0.2291 - val_accuracy: 0.1522 - val_loss: 0.3033
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3887 - loss: 0.2152 - val_accuracy: 0.1813 - val_loss: 0.3055
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4157 - loss: 0.2070 - val_accuracy: 0.2554 - val_loss: 0.3065
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - acc

In [12]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.3280 - loss: 1.6090 - val_accuracy: 0.0564 - val_loss: 0.3140
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3465 - loss: 0.2327 - val_accuracy: 0.1225 - val_loss: 0.3199
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3831 - loss: 0.2195 - val_accuracy: 0.2527 - val_loss: 0.2990
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4042 - loss: 0.2116 - val_accuracy: 0.3380 - val_loss: 0.2927
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4331 - loss: 0.2007 - val_accuracy: 0.4632 - val_loss: 0.2831
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4685 - loss: 0.1933 - val_accuracy: 0.3772 - val_loss: 0.2822
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4947 - loss: 0.1870 - val_accuracy: 0.4630 - val_loss: 0.2464
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accurac

In [13]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 151.17 seconds
  Test Time: 0.26 seconds
  Precision: [0.78313253 0.13442623 0.5        0.45070423 0.91062802]
  Recall: [0.52       0.75925926 0.5631068  0.60952381 0.75174477]
  F1-Score: [0.625      0.22841226 0.52968037 0.51821862 0.82359366]
  Confusion Matrix:
[[195 106  20   7  47]
 [  6  41   0   1   6]
 [ 13  20  58   4   8]
 [  2  19   7  64  13]
 [ 33 119  31  66 754]]
Week 2:
  Train Time: 178.16 seconds
  Test Time: 0.27 seconds
  Precision: [0.77738516 0.20207254 0.38953488 0.44936709 0.93884892]
  Recall: [0.58666667 0.72222222 0.65048544 0.67619048 0.78065803]
  F1-Score: [0.66869301 0.31578947 0.48727273 0.53992395 0.85247686]
  Confusion Matrix:
[[220  71  40  14  30]
 [  5  39   2   3   5]
 [  9  15  67   7   5]
 [  3  10  10  71  11]
 [ 46  58  53  63 783]]
Week 3:
  Train Time: 122.86 seconds
  Test Time: 0.23 seconds
  Precision: [0.76106195 0.21472393 0.45390071 0.59793814 0.92444444]
  Recall: [0.688  